In [1]:
import pandas as pd
import spacy
from spacytextblob.spacytextblob import SpacyTextBlob
from langdetect import detect_langs, LangDetectException
from geopy.geocoders import Nominatim
import pycountry

In [2]:
df1 = pd.read_pickle(
    '/Users/okankoklu/Desktop/EXETER/Year 3/BEM3064 Data Viz/DATAVIZ PROJECT/cleaned_tweets_dataframe.pkl')

display(df1)

,id,date,number_of_replies,number_of_retweets,number_of_likes,language,raw_content,longitude,latitude
0,1646333442651717633,2023-04-13 02:04:14+00:00,0,0,5,en,@user I noticed that electric car quit in the ...,-118.063298,33.791963
1,1646044706776793088,2023-04-12 06:56:54+00:00,1,0,5,en,Many of you ask me about my experience with my...,-3.952603,40.377576
2,1645731911191461894,2023-04-11 10:13:58+00:00,1,0,5,en,@user It's pennies mate £600 rent/mortgage £20...,-7.660857,54.633631
3,1645665048436805632,2023-04-11 05:48:17+00:00,0,3,14,en,@user Dreaming here as electric cars are not ...,152.668523,-27.767441
4,1645150581638438914,2023-04-09 19:43:58+00:00,0,0,3,en,"@user @user Looking good! Yeah, I do enjoy th...",-83.675290,36.540739
...,...,...,...,...,...,...,...,...,...
8889,1306479802925318145,2020-09-17 06:27:12+00:00,1,0,2,nl,@user Wij hebben een 30tal H2-wagens rondrijde...,4.172376,50.689144
8890,1607705706701983746,2022-12-27 11:51:24+00:00,3,0,9,nl,@user @user @user 1) Het is in de zomer drukke...,0.340613,44.619356
8891,1476542724983644165,2021-12-30 13:16:33+00:00,1,0,3,da,"@user @user @user Martin Grefte, de man die al...",7.492466,50.549509
8892,1427967707471925254,2021-08-18 12:16:46+00:00,5,0,9,nl,@user @user Het pijnpunt zit hem in het feit d...,6.464595,51.853814


## Sentiment Analysis using SpaCy

In [3]:
# Reverse geocoding

geolocator = Nominatim(user_agent="geoapiExercises")


def reverse_geocode(lat, lon):
    location = geolocator.reverse((lat, lon), exactly_one=True)
    if location is not None:
        address = location.raw['address']
        country_code = address.get('country_code', '')
        return country_code
    else:
        return None


df1['country_code'] = df1.apply(lambda row: reverse_geocode(
    row['latitude'], row['longitude']), axis=1, result_type='expand')

df1 = df1.dropna(subset='country_code')

display(df1)

,id,date,number_of_replies,number_of_retweets,number_of_likes,language,raw_content,longitude,latitude,country_code
0,1646333442651717633,2023-04-13 02:04:14+00:00,0,0,5,en,@user I noticed that electric car quit in the ...,-118.063298,33.791963,us
1,1646044706776793088,2023-04-12 06:56:54+00:00,1,0,5,en,Many of you ask me about my experience with my...,-3.952603,40.377576,es
2,1645731911191461894,2023-04-11 10:13:58+00:00,1,0,5,en,@user It's pennies mate £600 rent/mortgage £20...,-7.660857,54.633631,gb
3,1645665048436805632,2023-04-11 05:48:17+00:00,0,3,14,en,@user Dreaming here as electric cars are not ...,152.668523,-27.767441,au
4,1645150581638438914,2023-04-09 19:43:58+00:00,0,0,3,en,"@user @user Looking good! Yeah, I do enjoy th...",-83.675290,36.540739,us
...,...,...,...,...,...,...,...,...,...,...
8889,1306479802925318145,2020-09-17 06:27:12+00:00,1,0,2,nl,@user Wij hebben een 30tal H2-wagens rondrijde...,4.172376,50.689144,be
8890,1607705706701983746,2022-12-27 11:51:24+00:00,3,0,9,nl,@user @user @user 1) Het is in de zomer drukke...,0.340613,44.619356,fr
8891,1476542724983644165,2021-12-30 13:16:33+00:00,1,0,3,da,"@user @user @user Martin Grefte, de man die al...",7.492466,50.549509,de
8892,1427967707471925254,2021-08-18 12:16:46+00:00,5,0,9,nl,@user @user Het pijnpunt zit hem in het feit d...,6.464595,51.853814,de


In [4]:
# Converting country code to country names


def country_code_to_name(code):
    try:
        country = pycountry.countries.get(alpha_2=code.upper())
        return country.name
    except:
        return None


df1['country'] = df1['country_code'].apply(country_code_to_name)

df1 = df1.drop(columns='country_code')
df1 = df1.dropna(subset='country')

display(df1)

,id,date,number_of_replies,number_of_retweets,number_of_likes,language,raw_content,longitude,latitude,country
0,1646333442651717633,2023-04-13 02:04:14+00:00,0,0,5,en,@user I noticed that electric car quit in the ...,-118.063298,33.791963,United States
1,1646044706776793088,2023-04-12 06:56:54+00:00,1,0,5,en,Many of you ask me about my experience with my...,-3.952603,40.377576,Spain
2,1645731911191461894,2023-04-11 10:13:58+00:00,1,0,5,en,@user It's pennies mate £600 rent/mortgage £20...,-7.660857,54.633631,United Kingdom
3,1645665048436805632,2023-04-11 05:48:17+00:00,0,3,14,en,@user Dreaming here as electric cars are not ...,152.668523,-27.767441,Australia
4,1645150581638438914,2023-04-09 19:43:58+00:00,0,0,3,en,"@user @user Looking good! Yeah, I do enjoy th...",-83.675290,36.540739,United States
...,...,...,...,...,...,...,...,...,...,...
8889,1306479802925318145,2020-09-17 06:27:12+00:00,1,0,2,nl,@user Wij hebben een 30tal H2-wagens rondrijde...,4.172376,50.689144,Belgium
8890,1607705706701983746,2022-12-27 11:51:24+00:00,3,0,9,nl,@user @user @user 1) Het is in de zomer drukke...,0.340613,44.619356,France
8891,1476542724983644165,2021-12-30 13:16:33+00:00,1,0,3,da,"@user @user @user Martin Grefte, de man die al...",7.492466,50.549509,Germany
8892,1427967707471925254,2021-08-18 12:16:46+00:00,5,0,9,nl,@user @user Het pijnpunt zit hem in het feit d...,6.464595,51.853814,Germany


In [5]:
# Detecting languages


def detect_language(text):
    try:
        if len(text) > 0:
            lang_list = detect_langs(text)
            lang = lang_list[0].lang
            return lang
        else:
            return None
    except LangDetectException:
        return None

df1['detected_language'] = df1['raw_content'].apply(lambda x: detect_language(x))

df1 = df1.dropna(subset='detected_language')

lan_list = ['en', 'es', 'fr', 'it', 'pt','de', 'zh-cn', 'ru', 'ja', 'nl', 'ko']

df2 = df1.copy()

df2 = df2[df2['detected_language'].isin(lan_list)]

display(df2)


,id,date,number_of_replies,number_of_retweets,number_of_likes,language,raw_content,longitude,latitude,country,detected_language
0,1646333442651717633,2023-04-13 02:04:14+00:00,0,0,5,en,@user I noticed that electric car quit in the ...,-118.063298,33.791963,United States,en
1,1646044706776793088,2023-04-12 06:56:54+00:00,1,0,5,en,Many of you ask me about my experience with my...,-3.952603,40.377576,Spain,en
2,1645731911191461894,2023-04-11 10:13:58+00:00,1,0,5,en,@user It's pennies mate £600 rent/mortgage £20...,-7.660857,54.633631,United Kingdom,en
3,1645665048436805632,2023-04-11 05:48:17+00:00,0,3,14,en,@user Dreaming here as electric cars are not ...,152.668523,-27.767441,Australia,en
4,1645150581638438914,2023-04-09 19:43:58+00:00,0,0,3,en,"@user @user Looking good! Yeah, I do enjoy th...",-83.675290,36.540739,United States,en
...,...,...,...,...,...,...,...,...,...,...,...
8888,1172199659101724672,2019-09-12 17:25:51+00:00,3,3,20,de,"@user @user @user Und mit ""Verkehrswende"" mein...",13.088304,52.338079,Germany,de
8889,1306479802925318145,2020-09-17 06:27:12+00:00,1,0,2,nl,@user Wij hebben een 30tal H2-wagens rondrijde...,4.172376,50.689144,Belgium,nl
8890,1607705706701983746,2022-12-27 11:51:24+00:00,3,0,9,nl,@user @user @user 1) Het is in de zomer drukke...,0.340613,44.619356,France,nl
8892,1427967707471925254,2021-08-18 12:16:46+00:00,5,0,9,nl,@user @user Het pijnpunt zit hem in het feit d...,6.464595,51.853814,Germany,nl


In [6]:
# Adding NLP pipelines for all languages

english = spacy.load('en_core_web_sm')
chinese = spacy.load('zh_core_web_sm')
french = spacy.load('fr_core_news_sm')
german = spacy.load('de_core_news_sm')
italian = spacy.load('it_core_news_sm')
japanese = spacy.load('ja_core_news_sm')
portugese = spacy.load('pt_core_news_sm')
russian = spacy.load('ru_core_news_sm')
spanish = spacy.load('es_core_news_sm')
dutch = spacy.load('nl_core_news_sm')
korean = spacy.load('ko_core_news_sm')

for language in [english, chinese, french, german, italian, japanese, portugese, russian, spanish, dutch, korean]:
    language.add_pipe('spacytextblob')


display(df2)

,id,date,number_of_replies,number_of_retweets,number_of_likes,language,raw_content,longitude,latitude,country,detected_language
0,1646333442651717633,2023-04-13 02:04:14+00:00,0,0,5,en,@user I noticed that electric car quit in the ...,-118.063298,33.791963,United States,en
1,1646044706776793088,2023-04-12 06:56:54+00:00,1,0,5,en,Many of you ask me about my experience with my...,-3.952603,40.377576,Spain,en
2,1645731911191461894,2023-04-11 10:13:58+00:00,1,0,5,en,@user It's pennies mate £600 rent/mortgage £20...,-7.660857,54.633631,United Kingdom,en
3,1645665048436805632,2023-04-11 05:48:17+00:00,0,3,14,en,@user Dreaming here as electric cars are not ...,152.668523,-27.767441,Australia,en
4,1645150581638438914,2023-04-09 19:43:58+00:00,0,0,3,en,"@user @user Looking good! Yeah, I do enjoy th...",-83.675290,36.540739,United States,en
...,...,...,...,...,...,...,...,...,...,...,...
8888,1172199659101724672,2019-09-12 17:25:51+00:00,3,3,20,de,"@user @user @user Und mit ""Verkehrswende"" mein...",13.088304,52.338079,Germany,de
8889,1306479802925318145,2020-09-17 06:27:12+00:00,1,0,2,nl,@user Wij hebben een 30tal H2-wagens rondrijde...,4.172376,50.689144,Belgium,nl
8890,1607705706701983746,2022-12-27 11:51:24+00:00,3,0,9,nl,@user @user @user 1) Het is in de zomer drukke...,0.340613,44.619356,France,nl
8892,1427967707471925254,2021-08-18 12:16:46+00:00,5,0,9,nl,@user @user Het pijnpunt zit hem in het feit d...,6.464595,51.853814,Germany,nl


In [7]:
# Calculating the polarity scores (-1 to 1)
# -1 and 1 are most possible negative and positive values respectively

language_dict = {
    'en': english,
    'es': spanish,
    'fr': french,
    'it': italian,
    'pt': portugese,
    'de': german,
    'zh-cn': chinese,
    'ru': russian,
    'ja': japanese,
    'nl': dutch,
    'ko': korean
}


def polarity_calculator(df2):
    detected_language = df2['detected_language']
    lan = language_dict[detected_language]
    doc = lan(df2['raw_content'])
    polarity = doc._.polarity
    return polarity


df2['polarity_score'] = df2.apply(polarity_calculator, 1)

display(df2)


,id,date,number_of_replies,number_of_retweets,number_of_likes,language,raw_content,longitude,latitude,country,detected_language,polarity_score
0,1646333442651717633,2023-04-13 02:04:14+00:00,0,0,5,en,@user I noticed that electric car quit in the ...,-118.063298,33.791963,United States,en,0.000000
1,1646044706776793088,2023-04-12 06:56:54+00:00,1,0,5,en,Many of you ask me about my experience with my...,-3.952603,40.377576,Spain,en,0.500000
2,1645731911191461894,2023-04-11 10:13:58+00:00,1,0,5,en,@user It's pennies mate £600 rent/mortgage £20...,-7.660857,54.633631,United Kingdom,en,0.000000
3,1645665048436805632,2023-04-11 05:48:17+00:00,0,3,14,en,@user Dreaming here as electric cars are not ...,152.668523,-27.767441,Australia,en,0.133333
4,1645150581638438914,2023-04-09 19:43:58+00:00,0,0,3,en,"@user @user Looking good! Yeah, I do enjoy th...",-83.675290,36.540739,United States,en,0.478906
...,...,...,...,...,...,...,...,...,...,...,...,...
8888,1172199659101724672,2019-09-12 17:25:51+00:00,3,3,20,de,"@user @user @user Und mit ""Verkehrswende"" mein...",13.088304,52.338079,Germany,de,0.000000
8889,1306479802925318145,2020-09-17 06:27:12+00:00,1,0,2,nl,@user Wij hebben een 30tal H2-wagens rondrijde...,4.172376,50.689144,Belgium,nl,0.000000
8890,1607705706701983746,2022-12-27 11:51:24+00:00,3,0,9,nl,@user @user @user 1) Het is in de zomer drukke...,0.340613,44.619356,France,nl,0.000000
8892,1427967707471925254,2021-08-18 12:16:46+00:00,5,0,9,nl,@user @user Het pijnpunt zit hem in het feit d...,6.464595,51.853814,Germany,nl,0.000000


In [8]:
df2.to_pickle(
    '/Users/okankoklu/Desktop/EXETER/Year 3/BEM3064 Data Viz/DATAVIZ PROJECT/sentiment_analysis.pkl')